# Data

Data is handled by `Datasets` in the `revolver.data` module.

- `revolver.data.seg` defines the base segmentation dataset interface and masking datasets that load mask-wise instead of image-wise.
- `revolver.data.pascal` has datasets for VOC/SBDD semantic and instance segmentation.
- `revolver.data.sparse` has a wrapper for static and dynamic sparse targets.
- `revolver.data.filter` has wrappers for filtering and mapping targets.

The wrappers are compositional so that a given kind of data can be loaded by composing a base dataset with the necessary wrappers. This notebook illustrates common use cases.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

from revolver.data.pascal import VOCSemSeg, VOCInstSeg, SBDDSemSeg, SBDDInstSeg
from revolver.data.seg import MaskSemSeg, MaskInstSeg
from revolver.data.sparse import SparseSeg
from revolver.data.filter import TargetFilter, TargetMapper

Let's switch the working dir to the project root to align notebook usage with the rest of the code. (The project root is found through git.)

In [ ]:
import os
import subprocess

root_dir = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip()
os.chdir(root_dir)

Let's make a dataset and have a first look:

In [ ]:
# the default dataset root dir works on its own if invoked from the root dir of the project,
# and this call gives the path only as an illustration.
ds = VOCSemSeg(root_dir='./data/voc2012')  
print("Dataset with {} images in split {} and {} classes:\n{}".format(len(ds), ds.split, ds.num_classes, ds.classes))

# all datasets return a 3-tuple with image, target/label, and auxiliary dictionary
# note: image and target are np arrays unless they are cast by transforms
im, target, aux = ds[0]  

plt.figure()
plt.title("item ID {}".format(ds.slugs[0]))
plt.imshow(im)
plt.axis('off')
# plot the target class-by-class
# in the next step we'll make this simpler and prettier with a palette to color the classes
plt.figure(figsize=(16, 4))
plt.subplot(1, 3, 1)
plt.imshow(target == ds.classes.index('aeroplane'))
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(target == ds.classes.index('person'))
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(target == ds.ignore_index)
plt.axis('off')

Having done that let's make a helper to inspect the data.

In [ ]:
def load_and_show(ds, title, count=1, print_aux=True):
    for i in range(count):
        im, target, aux = ds[np.random.randint(0, len(ds))]
        im = Image.fromarray(im, mode='RGB')
        target = Image.fromarray(target, mode='P')
        target.putpalette(ds.palette)  # plot with pretty colors
        fig = plt.figure(figsize=(12, 12))
        plt.subplot(1, 2, 1)
        if print_aux:
            figtitle = title + " " + str(aux)
        plt.title(title)
        plt.imshow(im)
        plt.axis('off')
        plt.tight_layout()
        plt.subplot(1, 2, 2)
        plt.imshow(target)
        plt.axis('off')

Loading semantic and instance segmentation data from VOC and its extended annotations in SBDD.

In [ ]:
ds = VOCSemSeg()  # default split is train
load_and_show(ds, "VOC semantic segmentation")

ds = SBDDSemSeg(split='train')
load_and_show(ds, "SBDD semantic segmentation")

ds = VOCInstSeg(split='val')
load_and_show(ds, "VOC instance segmentation")

ds = SBDDInstSeg()
load_and_show(ds, "SBDD instance segmentation")

Masking decomposes the dataset over masks instead of images. Indexing mask-wise instead of image-wise makes it possible to sample fairly over all masks whether they are class or instance masks. Were the indexing over images, first sampling an image and then sampling a mask would sample masks inversely proportionally to the number of masks in the image.

These datasets return the **binary** segmentation of a single mask as the target along with class and instance indices in the auxiliary dict.

In [ ]:
ds = MaskSemSeg(VOCSemSeg())
load_and_show(ds, "VOC class masks", count=3)

# instance masking requires both a semantic seg. and an instance seg. dataset
# to preserve class information
ds = MaskInstSeg(VOCSemSeg(), VOCInstSeg())
load_and_show(ds, "VOC instance masks", count=3)

Sparsity makes targets that are spatially sparse. The `SparseSeg` wrapper takes a count and limits every target value, whether class or instance, to that many points. The sparsity can be dynamic (the default), and sampled on every load, or static, and sampled once and for all on init to simulate a fixed sparse dataset.

In [ ]:
# reduce masks to 100 points per class, resampled on every load
ds = VOCSemSeg()
ds = SparseSeg(ds, count=100)
load_and_show(ds, "sparse VOC semantic segmentation", count=3)

# reduce masks to a fixed 16 points per class
# re-loading does not choose new points
ds = VOCSemSeg()
ds = SparseSeg(ds, count=16, static=True)
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.title("static sparsity")
im, target, aux = ds[0]
plt.imshow(target)
plt.subplot(1, 2, 2)
im, target, aux = ds[0]
plt.imshow(target)

Filtering and mapping restricts and transforms targets.

In [ ]:
# filter classes to only load images that include the filtered classes
ds = VOCSemSeg()
classes_to_filter = (ds.classes.index('aeroplane'), ds.classes.index('train'), ds.classes.index('car'))
ds_filtered = TargetFilter(ds, classes_to_filter)
load_and_show(ds_filtered, "Planes, Trains, and Automobiles", count=3)

# filter classes to only load images that include the filtered classes, and exclude other classes from the target
ds_mapped = TargetMapper(ds_filtered, {k: k in classes_to_filter for k in range(len(ds.classes))})
load_and_show(ds_mapped, "Planes, Trains, and Automobiles ONLY", count=3, print_aux=False)

# collapse all classes to make foreground/background target 
ds_fgbg = TargetMapper(ds, {k: 1 for k in range(1, ds.num_classes + 1)})
load_and_show(ds_fgbg, "FG/BG", count=3, print_aux=False)

Now, let's combine the datasets and wrappers to make sparse instance masks of cats.

In [ ]:
ds = MaskInstSeg(VOCSemSeg(), VOCInstSeg())
ds = TargetFilter(ds, (ds.classes.index('cat'),))
ds = SparseSeg(ds, count=128)
load_and_show(ds, "sparse cat instance masks", count=12)

Note that `Datasets`, including compositional datasets that inherit from the `Wrapper` mixin such as the masking datasets, can be cached by pickling/unpickling. This can save a lot of time, as long as care is taken to fully decide the name for the cache file. Try running the cell below and compare it to the timing of the original instantiation above.

In [ ]:
import pickle

cache_path = 'notebooks/ds-cache.pkl'

pickle.dump(ds, open(cache_path, 'wb'))
del ds

ds = pickle.load(open(cache_path, 'rb'))
os.remove(cache_path)

load_and_show(ds, "sparse cat instance masks, cached", count=12)